**Nama:** Devin Augustin

**NIM:** 2440094352

In [ ]:
!pip install unidecode

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import numpy as np

import re, string
from unidecode import unidecode
import nltk
from nltk.tokenize import word_tokenize

#**Jawaban Nomor 3**

Diberikan suatu data teks yang berisi abstrak dari beberapa artikel (data_3-.csv),
Anda ingin mengelompokan topik dari abstrak-abstrak ini, namun anda belum memiliki label untuk setiap
abstraknya. Anda perlu menggunakan pendekatan yang tidak memerlukan label.Tentukan topik dari
abstrak tersebut dengan menggunakan data yang ada, lakukan eksperimen dengan beberapa jumlah topik,
lakukan analisa berdasarkan coherence score dan persona analysis terhadap beberapa sampel. Jelaskan
analisa anda!

In [ ]:
df3 = pd.read_csv("/content/data_3C.csv")

In [ ]:
df3.head()

,Unnamed: 0,ABSTRACT
0,200,The topological morphology--order of zeros a...
1,201,The purpose of this paper is to formulate an...
2,202,This paper considers the problem of autonomo...
3,203,This study explores the validity of chain ef...
4,204,Developing neural network image classificati...


In [ ]:
df3.drop(columns=['Unnamed: 0'], inplace=True)
df3.head(5)

,ABSTRACT
0,The topological morphology--order of zeros a...
1,The purpose of this paper is to formulate an...
2,This paper considers the problem of autonomo...
3,This study explores the validity of chain ef...
4,Developing neural network image classificati...


Data Preprocessing

In [ ]:
def cleansing(df):
    df_clean=df.str.lower() #mengubah semua teks menjadi huruf kecil
    df_clean=[re.sub(r"\d+","",i )for i in df_clean] #menghapus digit apa pun dalam teks dengan menggantinya dengan string kosong
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean] #menghapus karakter non-alphanumeric dengan menggantinya dengan spasi
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean] #menghilangkan spasi ekstra dengan mengganti beberapa spasi berturut-turut dengan satu spasi
    df_clean = [unidecode(i) for i in df_clean] #menghilangkan huruf yang memiliki aksen
    df_clean = [re.sub(r'[^a-zA-Z\s]', '', i) for i in df_clean] #menghilangkan non-english alphabet
    return df_clean

In [ ]:
clean_text = cleansing(df3['ABSTRACT']) #memanggil function 'cleansing' dengan menggunakan data kolom 'text' dari df dan mengembalikkan nilainya ke dalam variable 'clean_text'

In [ ]:
clean_text[0:5]

[' the topological morphology order of zeros at the positions of electrons with respect to a specific electron of laughlin state at filling fractions m m odd is homogeneous as every electron feels zeros of order m at the positions of other electrons although fairly accurate ground state wave functions for most of the other quantum hall states in the lowest landau level are quite well known it had been an open problem in expressing the ground state wave functions in terms of flux attachment to particles em a la this morphology of laughlin state with a very general consideration of flux particle relations only in spherical geometry we here report a novel method for determining morphologies of these states based on these we construct almost exact ground state wave functions for the coulomb interaction although the form of interaction may change the ground state wave function the same morphology constructs the latter irrespective of the nature of the interaction between electrons ',
 ' the

In [ ]:
#Tokenization
#Tokenization adalah proses memecah teks menjadi unit yang lebih kecil, seperti kata atau kalimat
word_list = [sentence.split() for sentence in clean_text]

In [ ]:
#Remove Stopwords
#Menghapus stopwords mengacu pada proses menghilangkan kata-kata umum (seperti "the", "is", "and") dari teks
#yang dianggap tidak penting untuk analisis atau tidak berkontribusi banyak pada keseluruhan makna.
from nltk.corpus import stopwords
nltk.download('stopwords')

list_stopwords=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
token_clean =[[word for word in sentence if not word in list_stopwords] for sentence in word_list]

In [ ]:
#Lemmatization
#Lemmatisasi adalah proses mereduksi kata menjadi bentuk dasarnya atau kamus (lemma) untuk mencapai analisis dan pemahaman teks yang lebih baik.
from nltk import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
final_token = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in token_clean]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
final_token[0]

['topological',
 'morphology',
 'order',
 'zero',
 'position',
 'electron',
 'respect',
 'specific',
 'electron',
 'laughlin',
 'state',
 'filling',
 'fraction',
 'odd',
 'homogeneous',
 'every',
 'electron',
 'feel',
 'zero',
 'order',
 'position',
 'electron',
 'although',
 'fairly',
 'accurate',
 'ground',
 'state',
 'wave',
 'function',
 'quantum',
 'hall',
 'state',
 'lowest',
 'landau',
 'level',
 'quite',
 'well',
 'known',
 'open',
 'problem',
 'expressing',
 'ground',
 'state',
 'wave',
 'function',
 'term',
 'flux',
 'attachment',
 'particle',
 'em',
 'la',
 'morphology',
 'laughlin',
 'state',
 'general',
 'consideration',
 'flux',
 'particle',
 'relation',
 'spherical',
 'geometry',
 'report',
 'novel',
 'method',
 'determining',
 'morphology',
 'state',
 'based',
 'construct',
 'almost',
 'exact',
 'ground',
 'state',
 'wave',
 'function',
 'coulomb',
 'interaction',
 'although',
 'form',
 'interaction',
 'may',
 'change',
 'ground',
 'state',
 'wave',
 'function',
 'morph

**Build Dictionary**

*Building dictionary* untuk *topic prediction* melibatkan pembuatan daftar kata atau frasa penting yang terkait dengan berbagai topik. Dengan menganalisis contoh teks dan menghitung frekuensi kata, kami memilih kata yang paling umum untuk setiap topik. Kata-kata ini membentuk kamus, dan ketika diterapkan pada teks baru, kita dapat memprediksi topik berdasarkan keberadaan kata kunci tersebut.

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from gensim import corpora, models

In [ ]:
#Membangun dictionary berdasarkan teks yang diberikan. Kamus ini akan digunakan untuk memetakan kata-kata dalam teks menjadi representasi numerik, seperti indeks atau ID kata-kata.d
def build_dic(text):
    dictionary = Dictionary(text)
    dictionary.filter_extremes(no_below=1, no_above=0.2, keep_n= 100000)
    return dictionary

In [ ]:
dictionary=build_dic(final_token)

In [ ]:
#Membangun vektor Bag of Words corpus berdasarkan teks yang diberikan dan kamus yang telah dibangun sebelumnya.
def build_vec(text,dictionary):
    bow_corpus = [dictionary.doc2bow(doc) for doc in text]
    return bow_corpus

In [ ]:
bow_corpus=build_vec(final_token,dictionary)

In [ ]:
#Menghitung score TF-IDF dari bag of words
def vector_tfidf(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    return corpus_tfidf

In [ ]:
corpus_tfidf=vector_tfidf(bow_corpus)

In [ ]:
#membangun model LDA untuk mengidentifikasi topik-topik yang ada dalam dokumen dan mendistribusikan kata-kata ke dalam topik-topik tersebut.
def model_lda(dictionary,bow_corpus,num_topic,alpha,eta):
    lda_model =  gensim.models.LdaMulticore(bow_corpus,
                                   num_topics = num_topic,
                                   id2word = dictionary,
                                   passes = 10,
                                   workers = 2,
                                   alpha=alpha,
                                   eta=eta)
    return lda_model

In [ ]:
lda_model=model_lda(dictionary,corpus_tfidf,4,0.7,0.7) #Disini mari kita coba menggunakan 4 jumlah topic

In [ ]:
#Fungsi untuk menemukan coherence score
#Coherence score digunakan untuk mengukur kualitas dan interpretabilitas topik yang dihasilkan oleh model pemodelan topik
def score_perf(lda_model,text,dictionary):
    coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dictionary, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return coherence_lda

In [ ]:
coherence_score=score_perf(lda_model,final_token,dictionary)

In [ ]:
print(coherence_score)

0.4928106611687548


Coherence score yang lebih tinggi menunjukkan bahwa kata-kata dalam suatu topik terkait lebih kuat dan koheren. Ini menunjukkan bahwa topiknya lebih dapat ditafsirkan dan mewakili tema yang berbeda.

In [ ]:
#Fungsi untuk mencetak topik-topik yang dihasilkan oleh model LDA beserta kata-kata yang termasuk dalam setiap topik.
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.000*"algorithm" + 0.000*"distribution" + 0.000*"q" + 0.000*"network" + 0.000*"state" + 0.000*"class" + 0.000*"architecture" + 0.000*"image" + 0.000*"quantum" + 0.000*"matrix"


Topic: 1 
Words: 0.000*"algorithm" + 0.000*"distribution" + 0.000*"network" + 0.000*"q" + 0.000*"quantum" + 0.000*"set" + 0.000*"matrix" + 0.000*"music" + 0.000*"state" + 0.000*"class"


Topic: 2 
Words: 0.000*"quantum" + 0.000*"algorithm" + 0.000*"distribution" + 0.000*"q" + 0.000*"network" + 0.000*"state" + 0.000*"music" + 0.000*"code" + 0.000*"set" + 0.000*"theorem"


Topic: 3 
Words: 0.001*"distribution" + 0.000*"algorithm" + 0.000*"network" + 0.000*"matrix" + 0.000*"state" + 0.000*"quantum" + 0.000*"structure" + 0.000*"function" + 0.000*"class" + 0.000*"set"




Setiap baris output menunjukkan informasi tentang satu topik. Kata-kata yang muncul setelah tanda "*" adalah kata-kata yang paling berkontribusi dalam membentuk topik tersebut, dan angka yang mengikuti kata-kata tersebut adalah bobotnya.

In [ ]:
#Melakukan cleansing terhadap data pada kolom 'ABSTRACT'
data_cleantest=cleansing(df3['ABSTRACT'])

In [ ]:
#Melakukan inferensi topik pada data menggunakan model LDA yang telah dilatih sebelumnya.
#Iterasi dilakukan melalui setiap dokumen dalam data dan untuk setiap dokumen, topik yang paling relevan dan kedua paling relevan ditentukan dengan memperhatikan skor probabilitasnya.
test_shape=len(data_cleantest)
topic1=[]
score1=[]
topic2=[]
score2=[]
sentence=[]
for doc in range(test_shape):
    sentence.append(data_cleantest[doc])
    topic1.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[0][0])
    score1.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[0][1])
    topic2.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[1][0])
    score2.append(sorted(lda_model[bow_corpus[doc]], key=lambda tup: -1*tup[1])[1][1])

In [ ]:
#Menggabungkan hasil inferensi topik dan skor probabilitasnya ke dalam satu DataFrame
sentence=pd.DataFrame(sentence,columns=['sentence']).reset_index()
topic1=pd.DataFrame(topic1,columns=['topic1']).reset_index()
score1=pd.DataFrame(score1,columns=['score1']).reset_index()
topic2=pd.DataFrame(topic2,columns=['topic2']).reset_index()
score2=pd.DataFrame(score2,columns=['score2']).reset_index()
test_result=pd.concat([sentence,topic1,score1,topic2,score2],axis=1)

In [ ]:
#Menunjukkan 20 hasil pertama
test_result=test_result[['sentence','topic1','score1','topic2','score2']]
test_result.head(20)

,sentence,topic1,score1,topic2,score2
0,the topological morphology order of zeros at ...,2,0.357501,3,0.322040
1,the purpose of this paper is to formulate and...,1,0.347985,2,0.258408
2,this paper considers the problem of autonomou...,3,0.308258,1,0.240056
3,this study explores the validity of chain eff...,0,0.268768,0,0.268615
4,developing neural network image classificatio...,0,0.376399,0,0.245255
5,we propose a new multi frame method for effic...,0,0.497046,3,0.184000
6,let k be an algebraically closed field of pos...,2,0.296733,1,0.264872
7,we present the mixed galerkin discretization ...,3,0.360869,1,0.246296
8,the regularity of earthquakes their destructi...,3,0.366661,2,0.254459
9,in this paper we prove some difference analog...,2,0.381611,1,0.288674


#**Jawaban Nomor 4**

Anda memiliki suatu teks (data.4-.txt) yang ingin anda simpulkan dengan
bantuan algoritma. Lakukanlah pemodelan text summarization menggunakan pendekatan extractive
dengan menggunakan top 3 scores.

In [ ]:
from nltk.cluster.util import cosine_distance

In [ ]:
#Fungsi untuk membaca file
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    #memisahkan kalimat berdasarkan pemisah titik (.).
    article = filedata[0].split(". ")
    sentences = []
    for sentence in article:
        print(sentence)
        #menghapus karakter non alfabet
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop()

    return sentences

In [ ]:
text = read_article('data_4C.txt')

The transfer of vocational education and training (VET) systems is currently an important topic in international debates (Davoine and Deitmer Citation2020; Maurer and Gonon Citation2014a; Valiente and Scandurra Citation2017; Allais Citation2010; King Citation2014; McGrath Citation2002)
The reasons given for policy transfer in the VET sector are complex and include aspects like (youth) unemployment reduction, poverty reduction, equipping skilled workers and increasing economic growth (Wiriadidjaja, Andriasanti, and Jane Citation2019; Stockmann Citation2019; France Diplomacy Citation2019; Bank et al
Citation2015)
However, surprisingly little recent research has explored issues such as practicability, successes achieved, problems encountered, and long-term effects
Scholars are more likely to focus on the theoretical basis for training transfer (Dolowitz and Marsh Citation2000), or on options for transfer at a more intermediate level of abstraction (Gonon Citation2014)
Policy transfer’ can

In [ ]:
#Fungsi untuk menghitung tingkat kemiripan antara dua kalimat berdasarkan vektorisasi kata-kata yang ada dalam kalimat tersebut.
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []

    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]

    all_words = list(set(sent1 + sent2))

    #Membangun vektor 0 dengan dimensi mengikuti len dari all_words
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    #Membangun vektor untuk kalimat pertama
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1

    #Membangun vektor untuk kalimat kedua
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

In [ ]:
#Fungsi untuk membangun matriks kemiripan antara kalimat-kalimat dalam suatu daftar.
def build_similarity_matrix(sentences, stop_words):
    #Membuat similarity matrix kosong
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #abaikan jika kalimat sama
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [ ]:
sentence_similarity_martix = build_similarity_matrix(text, stop_words=stopwords.words('english'))
sentence_similarity_martix

array([[0.        , 0.03005565, 0.        , 0.        , 0.13245324,
        0.        , 0.        , 0.        , 0.        , 0.04012862,
        0.14547859, 0.04811252, 0.06600984, 0.11111111, 0.        ,
        0.11547005, 0.06804138, 0.        , 0.        , 0.        ,
        0.03703704, 0.        , 0.06415003, 0.        , 0.        ,
        0.06804138, 0.        ],
       [0.03005565, 0.        , 0.        , 0.        , 0.07165744,
        0.05609927, 0.        , 0.08347839, 0.        , 0.09769344,
        0.11805627, 0.11713032, 0.08035074, 0.09016696, 0.        ,
        0.        , 0.05521576, 0.        , 0.        , 0.04173919,
        0.        , 0.        , 0.10411584, 0.        , 0.        ,
        0.11043153, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.35355339, 0.        , 0.    

In [ ]:
#Fungsi untuk menghasilkan skor PageRank untuk setiap kalimat
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
scores = nx.pagerank(sentence_similarity_graph)
scores

{0: 0.03273091522421835,
 1: 0.03666725498531675,
 2: 0.02400955938727758,
 3: 0.01805605209893996,
 4: 0.05061174265286081,
 5: 0.055382089622588206,
 6: 0.030355735096742772,
 7: 0.030961023532759067,
 8: 0.006134969325153374,
 9: 0.07932144283189654,
 10: 0.06856682994137366,
 11: 0.05433795109405862,
 12: 0.058360292719297074,
 13: 0.0657056670360068,
 14: 0.006134969325153374,
 15: 0.04510370780518802,
 16: 0.03241179332597175,
 17: 0.03424944229889421,
 18: 0.02385326766728013,
 19: 0.023472688977320296,
 20: 0.032077947178768065,
 21: 0.006134969325153374,
 22: 0.0589366944991609,
 23: 0.020946233388572714,
 24: 0.012394326582128409,
 25: 0.0713482514716912,
 26: 0.021734182606227823}

Kita dapat memperoleh skor PageRank yang menunjukkan tingkat pentingnya setiap kalimat dalam konteks kemiripan kalimat. Skor ini dapat digunakan untuk mengidentifikasi kalimat-kalimat yang lebih penting atau relevan dalam teks, seperti kalimat-kalimat yang paling mewakili topik atau kalimat-kalimat yang memiliki dampak lebih besar dalam pemahaman teks.

In [ ]:
#Fungsi untuk menghasilkan summary text
def generate_summary(file_name, top_n):
    stop_words = stopwords.words('english')
    summarize_text = []

    #Baca teks dan pisahkan
    sentences =  read_article(file_name)

    #Hasilkan Similarity Martix di seluruh kalimat
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    #Beri peringkat kalimat dalam kesamaan martiks
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    #Urutkan peringkat dan pilih kalimat teratas
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    print("\nIndexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    #Output dari summarize text
    print("\nSummarize Text: \n", ". ".join(summarize_text))


In [ ]:
generate_summary( "data_4C.txt", 3)

The transfer of vocational education and training (VET) systems is currently an important topic in international debates (Davoine and Deitmer Citation2020; Maurer and Gonon Citation2014a; Valiente and Scandurra Citation2017; Allais Citation2010; King Citation2014; McGrath Citation2002)
The reasons given for policy transfer in the VET sector are complex and include aspects like (youth) unemployment reduction, poverty reduction, equipping skilled workers and increasing economic growth (Wiriadidjaja, Andriasanti, and Jane Citation2019; Stockmann Citation2019; France Diplomacy Citation2019; Bank et al
Citation2015)
However, surprisingly little recent research has explored issues such as practicability, successes achieved, problems encountered, and long-term effects
Scholars are more likely to focus on the theoretical basis for training transfer (Dolowitz and Marsh Citation2000), or on options for transfer at a more intermediate level of abstraction (Gonon Citation2014)
Policy transfer’ can

**Hasil Text Summarization:**

Terms such as transfer, export, and policy borrowing or learning are also imprecisely defined or controversial, partly because different disciplines use different metaphors for the policy transfer process. The next sections introduce policy transfer research from various disciplines. Policy transfer is commonly used in the international and interdisciplinary context: this term covers the transfer of education, but it also implies the transfer of procedures, measures, strategies, and concepts in the broadest sense (Rose Citation1991)